In [1]:
from postgres_connect import *

In [17]:
path = 'tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)

Tunnel Started
Connection Made


In [18]:
query_path = 'rider_pip_closure.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
cursor = conn.cursor()
cursor.execute(custom_query)
conn.commit()
# conn.close()

In [6]:
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=columns)

In [7]:
df

,key,pip_date,rider_id,shipping_city,rider_name,hub,vendor_name,rider_age,need_improvement,starting_fasr,closing_fasr,pip_status,pip_result,update_time
0,20240122-Ranjeet_9205025720_05,2024-01-22,Ranjeet_9205025720_05,Delhi,Ranjeet JPS,DS DEL DWK,JPS,17,Morning COD(53%),73.0,51.0,closed,failed,2024-02-01 17:07:47.734436
1,20240122-Govinga_9910340433_021,2024-01-22,Govinga_9910340433_021,Delhi,Govinga PS,DS DEL RNGR,PS Manpower Solution LLP,32,Morning Prepaid (74%),73.0,72.0,closed,failed,2024-02-01 17:07:47.734436
2,20240122-Umesh_Kumar_Ps_16,2024-01-22,Umesh_Kumar_Ps_16,Delhi,Umesh Kumar Ps,DS DEL RNGR,PS Manpower,140,Morning COD(57%),74.0,81.0,closed,passed,2024-02-01 17:07:47.734436
3,20240122-Bala_8185059013_04,2024-01-22,Bala_8185059013_04,Hyderabad,Bala Krishna,DS HYD KMP,Peel Works,32,Morning Prepaid (74%) Morning Prepaid (33%),54.0,56.0,closed,failed,2024-02-01 17:07:47.734436
4,20240122-Narshimha_9703975750_04,2024-01-22,Narshimha_9703975750_04,Hyderabad,Narshimha,DS HYD KMP,Olive Green,28,Morning Prepaid (76%) Morning Prepaid (46%),66.0,NaN,closed,failed,2024-02-01 17:07:47.734436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,20240122-Javed_Kumte_3,2024-01-22,Javed_Kumte_3,Mumbai,Javed Kumte,DS BOM SWRI,DUMMY_1,142,Morning Prepaid (78%),73.0,77.0,closed,failed,2024-02-01 17:07:47.734436
71,20240122-Gurunath_bane_22,2024-01-22,Gurunath_bane_22,Mumbai,Gurunath bane,DS BOM SWRI,Team HR,55,Morning Prepaid (83%),79.0,72.0,closed,failed,2024-02-01 17:07:47.734436
72,20240122-Tabeesh_Shaikh_3,2024-01-22,Tabeesh_Shaikh_3,Mumbai,Tabeesh Shaikh,DS BOM SWRI,rapydex,82,Morning COD(45%),82.0,90.0,closed,passed,2024-02-01 17:07:47.734436
73,20240122-Satish_7386752667_04,2024-01-22,Satish_7386752667_04,Hyderabad,Satish,DS HYD TRNK,Blitz_Self,20,Morning Prepaid (62%),59.0,96.0,closed,passed,2024-02-01 17:07:47.734436


In [8]:
query_path = 'rider_pip_closure_performance.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_pip = pd.read_sql(custom_query,conn)
df_pip.head()

,pip_date,shipping_city,rider_id,rider_name,hub,performance_type,threshold,monday,tuesday,wednesday,thursday,friday,saturday,sunday,avg_fasr,status
0,2024-01-22,Bangalore,Bhaskar_S_2,Bhaskar R,DS BLR BOMM,Morning-Prepaid,0.85,1.00,1.00,0.90,1.00,0.87,0.83,0.92,0.94,pass
1,2024-01-22,Bangalore,Bhaskar_S_2,Bhaskar R,DS BLR BOMM,Morning-COD,0.60,0.73,0.66,0.44,0.78,0.71,0.83,0.80,0.68,pass
2,2024-01-22,Bangalore,Bhaskar_S_2,Bhaskar R,DS BLR BOMM,Evening-Prepaid,0.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
3,2024-01-22,Bangalore,Bhaskar_S_2,Bhaskar R,DS BLR BOMM,Evening-COD,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fail
4,2024-01-22,Bangalore,Harisha_N_2,Harisha N,DS BLR BOMM,Morning-Prepaid,0.85,1.00,1.00,1.00,1.00,0.90,1.00,1.00,0.98,pass


In [11]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Filter failed riders
failed_riders = df[df['pip_result'] == 'failed']

# Group by city and hub
grouped_data = failed_riders.groupby(['shipping_city', 'hub']).size().reset_index(name='failed_count')

# Group by city to get total rider count
total_riders = df.groupby(['shipping_city', 'hub']).size().reset_index(name='total_count')

# Merge the two dataframes on city
result_df = pd.merge(total_riders, grouped_data, on=['shipping_city','hub'], how='left')

# Calculate the failed rate without decimals
result_df['failed_rate'] = (result_df['failed_count'] / result_df['total_count']) * 100
result_df['failed_rate'] = result_df['failed_rate'].fillna(0)
result_df['failed_rate'] = result_df['failed_rate'].astype(int)

# Email configuration
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# Create the message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = "akshay@blitznow.in, sanjay@blitznow.in, david.kumar@blitznow.in, ajay.verma@blitznow.in, biswojit.jena@blitznow.in, chandrakanth.sunkoji@blitznow.in, rahul.lochav@blitznow.in, sourabh.vijay@blitznow.in, sourabh.vijayvergia@blitznow.in, syed.afrid@blitznow.in, anandbellary.hindushree@blitznow.in, dilip.kumar@blitznow.in, ma.saifullah@blitznow.in, shankar.poddar@blitznow.in, sumit.kamble@blitznow.in, anil@blitznow.in"
message['Subject'] = f"Rider PIP Closure Report Week - {failed_riders['pip_date'].max()}"

# Create an Excel file with two sheets
excel_file_path = 'failed_riders_report.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the failed_riders DataFrame to the first sheet
    failed_riders.to_excel(writer, sheet_name='Failed_Riders_Data', index=False)

    # Write the df_pip DataFrame to the second sheet
    df_pip.to_excel(writer, sheet_name='Additional_Sheet', index=False)

# Attach the Excel file
with open(excel_file_path, 'rb') as excel_file:
    attachment = MIMEApplication(excel_file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename='pip_closure_report.xlsx')
    message.attach(attachment)

# Create an HTML table for the email body with formatted headers
html_table = result_df.to_html(index=False, border=1, classes='table', escape=False, justify='center')
html_table = html_table.replace('<th>', '<th style="font-weight:bold; background-color:lightblue; text-align:center;">').replace('_', ' ').upper()

# Add the table to the email body
body = f"""\
<html>
  <head></head>
  <body>
    <p>Dear recipients,</p>
    <p>Please find attached the report for RIDER PIP Result for the week - {failed_riders['pip_date'].max()}.</p>
    <p>Summary:</p>
    {html_table}
    <p>Best regards,<br>Your Name</p>
  </body>
</html>
"""

message.attach(MIMEText(body, 'html'))

# Connect to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipients, message.as_string())

print("Email sent successfully.")

Email sent successfully.


In [16]:
conn.close()